In [1]:
## last gen 
from scipy.stats import spearmanr, pearsonr, kendalltau

import pandas as pd
import pickle
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

/tmp/ipykernel_443869/2430059226.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
#import allel
#vcf = allel.read_vcf('../key_files/greneNet_final_v1.1_LDpruned.recode.vcf')
#ecotypes_names = vcf['samples']

In [3]:
#pd.Series(ecotypes_names).to_csv('../key_files/founder_ecotype_names.csv', index=None)

In [4]:
ecotypes_names = pd.read_csv('../key_files/founder_ecotype_names.csv')['0'].to_list()

In [5]:
clim_sites_during_exp = pd.read_csv('../key_files/bioclimvars_experimental_sites_era5.csv')
#clim_sites_during_exp = pd.read_csv('../key_files/bioclimvars_sites_era5_year_2018.csv')


#sites_test = pd.Series(test).str.split('_').str[0].astype(int)

In [6]:
unique_sites = pd.read_csv('../key_files/final_gen.csv')['site'].unique()

In [7]:
#sort_bio1 = clim_sites_during_exp
clim_sites_during_exp_s = clim_sites_during_exp[clim_sites_during_exp['site'].isin(unique_sites)]

sort_bio1 = clim_sites_during_exp_s[['site', 'bio1']].sort_values('bio1')['site'].astype(str).values

In [8]:
sort_bio1_int = clim_sites_during_exp_s[['site', 'bio1']].sort_values('bio1')['site'].values

In [9]:
final_gen = pd.read_csv('../key_files/final_gen.csv')['sample_name'].to_list()
delta_ef=pd.read_csv('../key_files/delta_ecotype_freq.txt', sep = '\t', usecols = final_gen)#[['log_p1_p0','ecotype', 'site','plot']]

ef=pd.read_csv('../key_files/merged_ecotype_frequency.txt', sep = '\t', usecols = final_gen)#[['log_p1_p0','ecotype', 'site','plot']]
delta_ef.index = ecotypes_names
ef.index = ecotypes_names

In [10]:
unique_sites = pd.read_csv('../key_files/final_gen.csv')['site'].unique()

In [11]:
unique_sites

array([ 1,  2,  4,  5,  6,  9, 10, 11, 12, 13, 23, 24, 25, 26, 27, 28, 32,
       33, 37, 42, 43, 45, 46, 48, 49, 52, 53, 54, 55, 57, 60])

In [12]:
file_path = '../jacknife_lfmm/splits_samples_last_gen_no33.pkl'

# Open and load the .pkl file
with open(file_path, 'rb') as file:
    samples = pickle.load(file)

In [ ]:
## recalculate GO

In [19]:
genotype_counts = pd.read_csv('../leave_1_out/genotype_counts_fullgenome231ecotypes.csv').drop('Unnamed: 0', axis=1)

In [20]:
snp_dict = pd.read_csv('../key_files/var_pos_grenenet.csv')

In [21]:
split

NameError: name 'split' is not defined

{'0_1': [1048635, 1048634],
 '1_1': [1048635, 1048634],
 '2_1': [1048635, 1048634],
 '3_1': [1048635, 1048633],
 '4_1': [1048635, 1048634],
 '5_1': [1048635, 1048634],
 '6_1': [1048635, 1048634],
 '7_1': [1048635, 1048634],
 '8_1': [1048635, 1048634],
 '9_1': [1048635, 1048634],
 '10_1': [1048635, 1048634],
 '11_1': [1048635, 1048634],
 '12_1': [1048635, 1048634],
 '13_1': [1048635, 1048634],
 '14_1': [1048635, 1048634],
 '15_1': [1048635, 1048633],
 '16_1': [1048635, 1048634],
 '17_1': [1048635, 1048624],
 '18_1': [1048635, 1048634],
 '19_1': [1048635, 1048634],
 '20_1': [1048635, 1048634],
 '21_1': [1048635, 1048634],
 '22_1': [1048635, 1048631],
 '23_1': [1048635, 1048634],
 '24_1': [1048635, 1048634],
 '25_1': [1048635, 1048634],
 '26_1': [1048635, 1048634],
 '27_1': [1048635, 1048633],
 '28_1': [1048635, 1048634],
 '29_1': [1048635, 1048632],
 '0_0.8': [1048635, 838908],
 '1_0.8': [1048635, 838908],
 '2_0.8': [1048635, 838905],
 '3_0.8': [1048635, 838908],
 '4_0.8': [1048635, 8389

In [ ]:
snp_number = {}
for i in [1,0.8,0.5,0.3,0.1,0.01, 0.001]:
    print(i)
    go_per_site = {}
    for split in range(len(samples)):
        print(split)
        br_results = pd.read_csv(f'results/split_{split}/binomial_reg_results_last_gen.csv')
        il = len(br_results)
        br_results = br_results[br_results['pvalue'] < br_results['pvalue'].quantile(i)]
        snp_number[str(split) + '_' + str(i)] = [il, len(br_results)]


In [54]:
snp_number = pd.DataFrame(snp_number).T.reset_index()

In [60]:
snp_number['split'] = snp_number['index'].str.split('_').str[0]
snp_number['quantile'] = snp_number['index'].str.split('_').str[1]

In [62]:
snp_number = snp_number.drop('index',axis=1)

In [66]:
snp_number.columns = ['total_snps', 'snps_used', 'split', 'quantile']

In [68]:
aprox_total_used = snp_number.groupby('quantile')['snps_used'].mean()

In [70]:
aprox_total_used.reset_index()

,quantile,snps_used
0,0.001,1.046933e+03
1,0.01,1.048300e+04
2,0.1,1.048604e+05
3,0.3,3.145844e+05
4,0.5,5.243125e+05
5,0.8,8.389012e+05
6,1,1.048633e+06


In [17]:
br_results = pd.read_csv(f'results/split_1/binomial_reg_results_last_gen.csv')

In [28]:
len(samples)

30

In [ ]:
for i in [1]: #,0.8,0.5,0.3,0.1,0.01, 0.001]:
    go_per_site = {}
    for split in range(len(samples)):
        #print(split)
        br_results = pd.read_csv(f'results/split_{split}/binomial_reg_results_last_gen.csv')
        #print(len(br_results))
        #br_results = br_results[br_results['pvalue'] < 0.05/len(br_results)]
    
        br_results = br_results[br_results['pvalue'] < br_results['pvalue'].quantile(i)]
        #print(len(br_results))
        br_results = br_results.reset_index(drop=True)
    
        mask_full_genotype = snp_dict['id'].isin(br_results['snp_id'])
        
        genotype_counts_red = genotype_counts.loc[mask_full_genotype,:]
        
        genotype_counts_red = genotype_counts_red.reset_index(drop=True)
        genotype_counts_red = genotype_counts_red.replace({2: 1, 1: 0.5, 0: 0})
        
        site_predicted = br_results.columns[0]
        print(site_predicted)
        site_predict_proba = br_results[site_predicted]
    
        ## for each site, subrstract the actual value by the probability fo getting a success 
        genotype_counts_subs = genotype_counts_red.sub(site_predict_proba, axis=0)
        genotype_counts_subs = genotype_counts_subs**2
        ## ge tthe absolute value 
        
        ## sum over ecotypes 
        go = (genotype_counts_subs.sum(axis=0)) / len(genotype_counts_subs)
        go_per_site[site_predicted] = go
        #print(site_predicted)
    go_per_site_df = pd.DataFrame(go_per_site)
    
    go_per_site_df.index = go_per_site_df.index.astype(int)
    
    go_per_site_df.index = ecotypes_names
    
    #go_per_site_df.to_csv(f'genomic_offset_per_site_last_gen_prob_squared_l1out_quantile{i}.csv',index=None)


In [32]:
ecotypes_names

[10002,
 10006,
 10011,
 10013,
 10014,
 159,
 265,
 5151,
 5165,
 5768,
 5772,
 5784,
 5811,
 6013,
 6025,
 6040,
 6073,
 6074,
 6108,
 6150,
 6177,
 6180,
 6184,
 6188,
 6195,
 6209,
 6216,
 6243,
 6244,
 6898,
 6911,
 6915,
 6929,
 6932,
 6938,
 6940,
 6945,
 6958,
 6961,
 6963,
 6979,
 6987,
 7000,
 7002,
 7003,
 7008,
 7013,
 7025,
 7028,
 7031,
 7036,
 7062,
 7063,
 7067,
 7071,
 7077,
 7092,
 7103,
 7106,
 7125,
 7126,
 7127,
 7143,
 7164,
 7165,
 7186,
 7203,
 7209,
 7217,
 7218,
 7244,
 7255,
 7268,
 7273,
 7276,
 7282,
 7287,
 7288,
 7296,
 7298,
 7316,
 7323,
 7333,
 7346,
 7347,
 7353,
 7372,
 7378,
 7384,
 7394,
 7404,
 7411,
 7521,
 763,
 765,
 766,
 768,
 772,
 8214,
 8230,
 8231,
 8240,
 8247,
 8249,
 8311,
 8312,
 8351,
 8354,
 8357,
 8376,
 8387,
 9057,
 9058,
 9323,
 9371,
 9394,
 9416,
 9427,
 9470,
 9481,
 9506,
 9507,
 9510,
 9512,
 9517,
 9518,
 9521,
 9522,
 9523,
 9524,
 9526,
 9527,
 9528,
 9529,
 9534,
 9535,
 9537,
 9539,
 9542,
 9544,
 9547,
 9548,
 9549,
 

In [31]:
pd.read_csv('genomic_offset_per_site_last_gen_prob_squared_l1out_quantile1.csv')

,1,2,4,5,6,9,10,11,12,13,...,45,46,48,49,52,53,54,55,57,60
0,0.134432,0.133528,0.133304,0.132571,0.133631,0.133762,0.135316,0.133506,0.133577,0.135318,...,0.133495,0.134258,0.135079,0.133996,0.133767,0.134164,0.133811,0.134060,0.133765,0.134391
1,0.135435,0.134573,0.135206,0.134445,0.134443,0.134881,0.135818,0.134616,0.134762,0.135490,...,0.134539,0.135290,0.136386,0.134973,0.135003,0.134932,0.135004,0.135215,0.134942,0.135130
2,0.142458,0.148046,0.150108,0.150298,0.146751,0.144546,0.156080,0.145900,0.145001,0.154909,...,0.147463,0.142672,0.141004,0.143525,0.143382,0.144034,0.144504,0.143264,0.144289,0.153045
3,0.134749,0.132613,0.132894,0.132434,0.132997,0.133663,0.132683,0.133135,0.133436,0.132619,...,0.132831,0.134176,0.135964,0.133935,0.133682,0.133643,0.133907,0.134371,0.133793,0.132400
4,0.135577,0.133267,0.133508,0.132980,0.133748,0.134445,0.133133,0.133859,0.134203,0.133111,...,0.133518,0.134962,0.136904,0.134737,0.134518,0.134461,0.134698,0.135237,0.134580,0.132917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,0.164514,0.158256,0.160160,0.158176,0.158858,0.162157,0.152662,0.160531,0.161573,0.152570,...,0.158766,0.163028,0.167550,0.162950,0.163510,0.162211,0.162053,0.163671,0.162391,0.154364
227,0.127446,0.129354,0.129863,0.129546,0.128827,0.127961,0.134109,0.128430,0.128099,0.133690,...,0.128850,0.128248,0.127443,0.127872,0.127558,0.127896,0.127920,0.127589,0.127909,0.132056
228,0.126344,0.125709,0.126777,0.125941,0.125452,0.125936,0.127299,0.125768,0.125866,0.126843,...,0.125771,0.125853,0.127261,0.126045,0.126386,0.125913,0.125850,0.126343,0.125952,0.126561
229,0.130355,0.132569,0.133598,0.133129,0.131835,0.131032,0.137549,0.131575,0.131237,0.136791,...,0.132201,0.130464,0.130250,0.130779,0.130923,0.131009,0.130973,0.130583,0.130949,0.135471


In [ ]:
# genomic_offset_per_site_last_gen_prob_squared_l1out
# combined_results_onlysign_p_top_ecotypes.csv

In [ ]:
## start here  # genomic_offset_per_site_last_gen_prob_squared_l1out

In [ ]:
go_per_site_df = pd.read_csv('genomic_offset_per_site_last_gen_prob_squared_l1out.csv')

In [ ]:
for quantile in [1,0.8,0.5,0.3,0.1,0.01, 0.001]:
    
    go_per_site_df = pd.read_csv(f'genomic_offset_per_site_last_gen_prob_squared_l1out_quantile{quantile}.csv')
    
    go_per_site_df.index = ecotypes_names
    
    results = {}
    for site in unique_sites:
        if site != 33:
            ## one site 
            ## retreive ecotype frequecnies 
            ef1 = ef[[col for col in ef.columns if col.startswith(f'{site}_')]]
            cd = go_per_site_df[str(site)]
            
            for sample in ef1.columns:
                ef_f = ef1[sample]
                conc = pd.concat([ef_f, cd],axis=1)
                conc = conc.sort_values(sample)
                ## becasue some ecotpyes are nan 
                conc = conc.dropna()
                X_ranked = conc[sample]
                y_ranked = conc[str(site)]
        
                sp_correlation, _ = spearmanr(X_ranked, y_ranked)
                pearsonr_value = pearsonr(X_ranked, y_ranked)[0]
                kendall_tau, _ = kendalltau(X_ranked, y_ranked)
    
                X = conc[str(site)].values.reshape(-1, 1)  # Independent variable
                y = conc[sample].values # Dependent variable
        
                # Fit the linear model
                model = LinearRegression().fit(X, y)
                # Predict the valu.es
                y_pred = model.predict(X)
                
                r_squared = r2_score(y, y_pred)
        
                res_list = [sp_correlation, pearsonr_value, r_squared, kendall_tau]
        
                for i in [10,20,30,40,50, 60, 100]:
                    top_10_observed_indices = conc[sample].sort_values().tail(i).index
                    top_10_predicted_indices = conc[str(site)].sort_values().head(i).index
            
                    # Compare top 10 indices
                    common_top_indices = len(set(top_10_observed_indices).intersection(set(top_10_predicted_indices)))
            
                    # Calculate the percentage of common top 10 indices
                    top_10_match_percentage = (common_top_indices / i) * 100
                    res_list.append(top_10_match_percentage)
                results[sample] = res_list
    
    results = pd.DataFrame(results).T
    
    results.columns = ['sp_correlation', 'pearsonr', 'r_squared','kendall_tau', 'top_10','top_20','top_30','top_40','top_50', 'top_60', 'top_100']
    
    results['sp_correlation'] = results['sp_correlation'] *-1
    
    results['pearsonr'] = results['pearsonr']  *-1
    results['kendall_tau'] = results['kendall_tau']  *-1
    results = results.reset_index()
    
    results['site'] = results['index'].str.split('_').str[0]
    
    results['plot'] = results['index'].str.split('_').str[2]
    print(quantile)
    results.to_csv(f'results_{quantile}.csv',index=None)

In [ ]:
all = {}
for quantile in [1,0.8,0.5,0.3,0.1,0.01, 0.001]:
    data = pd.read_csv(f'results_{quantile}.csv',)
    all[quantile] = data

In [ ]:
all = pd.concat(all).reset_index()

In [ ]:
all.columns = ['quantile_snps_used', 'level_1', 'sample', 'sp_correlation', 'pearsonr',
       'r_squared', 'kendall_tau', 'top_10', 'top_20', 'top_30', 'top_40',
       'top_50', 'top_60', 'top_100', 'site', 'plot']

In [ ]:
all = all.drop(['level_1', 'top_10', 'top_20', 'top_30', 'top_40',
       'top_50', 'top_60', 'top_100'],axis=1)

In [ ]:
all

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a figure and a single axis
fig, ax = plt.subplots()

# Create the strip plot on the axis
sns.stripplot(data=all, y='sp_correlation', x='quantile_snps_used', ax=ax, color='grey', alpha=0.3, s=7.5, zorder=1)

# Overlay the box plot on the same axis
sns.boxplot(data=all, y='sp_correlation', x='quantile_snps_used', fill=False,showfliers=False, color='black', width=0.5, zorder=2, ax=ax)

# Add a horizontal line at y=0
ax.axhline(0, color='grey', linestyle='--')
#ax.set_title('Climatic Distance and Genomic Offset')

# Set properties for the axis
ax.grid(axis='y')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

plt.xlabel('Quantile of SNPs included in GO calculation')
plt.ylabel('Spearman Correlation')


# Adjust layout
plt.tight_layout()
plt.savefig('Quantile_snps_corr_go.png')
plt.show()


In [ ]:

# Add grid to the background for y-axis
plt.grid(True, which='major', axis='y', linestyle='--', linewidth=0.7)

# Remove all spines
sns.despine(left=True, right=True, top=True, bottom=True)

ax = sns.stripplot(data=combined_results, y='sp_correlation', x='dataset', order=order, color='grey', alpha=0.3, s=7.5, zorder=1)
sns.boxplot(data=combined_results, y='sp_correlation', x='dataset', order=order, showfliers=False, color='black', width=0.5, zorder=2, fill=False)

In [ ]:
from scipy.stats import spearmanr
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from scipy.stats import pearsonr

In [ ]:
results_mean = {}
for site in unique_sites:
    if site != 33:
        ## one site 
        ## retreive ecotype frequecnies 
        ef1 = ef[[col for col in ef.columns if col.startswith(f'{site}_')]]
        ef_mean = ef1.mean(axis=1)
        ef_mean.name = 'mean'
        cd = go_per_site_df[str(site)]
        
        conc = pd.concat([ef_mean, cd],axis=1)
        #conc = conc.sort_values('mean')
        ## becasue some ecotpyes are nan 
        conc = conc.dropna()
        X_ranked = conc['mean']
        y_ranked = conc[str(site)]

        sp_correlation, _ = spearmanr(X_ranked, y_ranked)
        pearsonr_value = pearsonr(X_ranked, y_ranked)[0]
        kendall_tau, _ = kendalltau(X_ranked, y_ranked)

        X = conc[str(site)].values.reshape(-1, 1)  # Independent variable
        y = conc['mean'].values # Dependent variable

        # Fit the linear model
        model = LinearRegression().fit(X, y)
        # Predict the valu.es
        y_pred = model.predict(X)
        
        r_squared = r2_score(y, y_pred)

        res_list = [sp_correlation, pearsonr_value, r_squared, kendall_tau]
        for i in [10,20,30,40,50]:
            ## tail on ecotypes because the largest go on bottom and head for go because the smalles go teh better/
            top_10_observed_indices =  conc['mean'].sort_values().tail(i).index
            top_10_predicted_indices = conc[str(site)].sort_values().head(i).index
    
            # Compare top 10 indices
            common_top_indices = len(set(top_10_observed_indices).intersection(set(top_10_predicted_indices)))
    
            # Calculate the percentage of common top 10 indices
            top_10_match_percentage = (common_top_indices / i) * 100
            res_list.append(top_10_match_percentage)
        results_mean[str(site)] = res_list

In [ ]:
results_mean = pd.DataFrame(results_mean).T

results_mean.columns = ['sp_correlation', 'pearsonr', 'r_squared','kendall_tau', 'top_10','top_20','top_30','top_40','top_50']

results_mean['sp_correlation'] = results_mean['sp_correlation'] * -1

results_mean['pearsonr'] = results_mean['pearsonr']  * -1
results_mean['kendall_tau'] = results_mean['kendall_tau']  * -1

results_mean = results_mean.reset_index()

results_mean = results_mean.rename(columns = {'index':'site'})

In [ ]:
results = {}
for site in unique_sites:
    if site != 33:
        ## one site 
        ## retreive ecotype frequecnies 
        ef1 = ef[[col for col in ef.columns if col.startswith(f'{site}_')]]
        cd = go_per_site_df[str(site)]
        
        for sample in ef1.columns:
            ef_f = ef1[sample]
            conc = pd.concat([ef_f, cd],axis=1)
            conc = conc.sort_values(sample)
            ## becasue some ecotpyes are nan 
            conc = conc.dropna()
            X_ranked = conc[sample]
            y_ranked = conc[str(site)]
    
            sp_correlation, _ = spearmanr(X_ranked, y_ranked)
            pearsonr_value = pearsonr(X_ranked, y_ranked)[0]
            kendall_tau, _ = kendalltau(X_ranked, y_ranked)

            X = conc[str(site)].values.reshape(-1, 1)  # Independent variable
            y = conc[sample].values # Dependent variable
    
            # Fit the linear model
            model = LinearRegression().fit(X, y)
            # Predict the valu.es
            y_pred = model.predict(X)
            
            r_squared = r2_score(y, y_pred)
    
            res_list = [sp_correlation, pearsonr_value, r_squared, kendall_tau]
    
            for i in [10,20,30,40,50, 60, 100]:
                top_10_observed_indices = conc[sample].sort_values().tail(i).index
                top_10_predicted_indices = conc[str(site)].sort_values().head(i).index
        
                # Compare top 10 indices
                common_top_indices = len(set(top_10_observed_indices).intersection(set(top_10_predicted_indices)))
        
                # Calculate the percentage of common top 10 indices
                top_10_match_percentage = (common_top_indices / i) * 100
                res_list.append(top_10_match_percentage)
            results[sample] = res_list

results = pd.DataFrame(results).T

results.columns = ['sp_correlation', 'pearsonr', 'r_squared','kendall_tau', 'top_10','top_20','top_30','top_40','top_50', 'top_60', 'top_100']

results['sp_correlation'] = results['sp_correlation'] *-1

results['pearsonr'] = results['pearsonr']  *-1
results['kendall_tau'] = results['kendall_tau']  *-1
results = results.reset_index()

results['site'] = results['index'].str.split('_').str[0]

results['plot'] = results['index'].str.split('_').str[2]

In [ ]:
results['site'].nunique()

In [ ]:
results

In [ ]:
# Calculate medians of 'sp_correlation' for 'results' only to determine the order
medians_results = results.groupby('site')['sp_correlation'].mean().sort_values()

# Set the size of the plot
plt.figure(figsize=(16, 6))

# Create the violin plot, using 'hue' to distinguish between 'results' and 'results_cd'
sns.boxplot(data=results, y='sp_correlation', x='site', order=medians_results.index)

# Draw a horizontal line at y=0
plt.axhline(0, color='grey', linestyle='--')

# Grid and spine settings
plt.grid(axis='y')
ax = plt.gca()  # Get current axes
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add labels and title
plt.xlabel('Site')


# Display the plot
plt.show()

In [ ]:
cd_results_mean = pd.read_csv('../climate_distance/naive_climate_distance_top_ecotypes_mean.csv')

In [ ]:
results_mean['site'] = results_mean['site'].astype(int)
# Add a new column to differentiate between the two DataFrames
results_mean['dataset'] = 'genomic_offset'
cd_results_mean['dataset'] = 'climatic_distance'
# Concatenate the two DataFrames
combined_results_mean = pd.concat([results_mean, cd_results_mean])

In [ ]:
cd_results = pd.read_csv('../climate_distance/naive_climate_distance_top_ecotypes.csv')

In [ ]:
#cd_results_mean = pd.read_csv('../climate_distance/climate_distance_results_mean_leave_1_out.csv')

In [ ]:
#cd_results = pd.read_csv('../climate_distance/naive_climate_distance_top_ecotypes.csv')

In [ ]:
cd_model = pd.read_csv('../climate_distance/climate_distance_model_results_leave_1_out_ecotype_freq_top.csv')

In [ ]:
baseline_pairwise_topecotypes_samples = pd.read_csv('baseline_pairwise_topecotypes_samples_top.csv')

In [ ]:
results['site'] = results['site'].astype(int)
baseline_pairwise_topecotypes_samples['site'] = baseline_pairwise_topecotypes_samples['site'].astype(int)
# Add a new column to differentiate between the two DataFrames
results['dataset'] = 'genomic_offset'
cd_results['dataset'] = 'climatic_distance'

cd_model['dataset'] = 'climatic_distance_model'
baseline_pairwise_topecotypes_samples['dataset'] = 'baseline_pairwise_samples'

In [ ]:
combined_results = pd.concat([results, cd_results, baseline_pairwise_topecotypes_samples, cd_model],axis=0)  # ,baseline_pairwise_topecotypes_samples

In [ ]:
combined_results = combined_results.reset_index(drop=True)

In [ ]:
# Calculate medians of 'sp_correlation' for 'results' only to determine the order
medians_results = cd_results.groupby('site')['kendall_tau'].mean().sort_values()

# Set the size of the plot
plt.figure(figsize=(16, 6))

# Create the violin plot, using 'hue' to distinguish between 'results' and 'results_cd'
sns.boxplot(data=combined_results, y='kendall_tau', x='site', hue='dataset', order=medians_results.index)

# Draw a horizontal line at y=0
plt.axhline(0, color='grey', linestyle='--')

# Grid and spine settings
plt.grid(axis='y')
ax = plt.gca()  # Get current axes
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add labels and title
plt.xlabel('Site')


# Display the plot
plt.show()

In [ ]:
# Calculate medians of 'sp_correlation' for 'results' only to determine the order
medians_results = cd_results.groupby('site')['kendall_tau'].mean().sort_values()

# Set the size of the plot
plt.figure(figsize=(16, 6))

# Create the violin plot, using 'hue' to distinguish between 'results' and 'results_cd'
sns.boxplot(data=combined_results, y='kendall_tau', x='site', hue='dataset', order=medians_results.index)

# Draw a horizontal line at y=0
plt.axhline(0, color='grey', linestyle='--')

# Grid and spine settings
plt.grid(axis='y')
ax = plt.gca()  # Get current axes
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add labels and title
plt.xlabel('Site')


# Display the plot
plt.show()

In [ ]:
#combined_results.to_csv('combined_results_onlysign_p_top_ecotypes.csv',index=None)

In [ ]:
combined_results.head(1)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

variable = 'kendall_tau'
order_touse = sort_bio1_int

# Define a custom color palette mapping each dataset to a specific color
palette = {
    'climatic_distance': 'white',  # Light green
    'genomic_offset': '#32CD32',     # Dark green
    'climatic_distance_model': 'grey',  # Lime green
    'baseline_pairwise_samples': 'black' # Forest green
}


# Create 4 subplots (4 rows, 1 column)
fig, axes = plt.subplots(6, 1, figsize=(12, 26), sharex=True, sharey=True)

# Plot 1: Only 'climatic distance'
sns.boxplot(data=combined_results[combined_results['dataset'] == 'climatic_distance'], order=order_touse,
            y=variable, x='site', hue='dataset', ax=axes[0], palette=palette)
axes[0].axhline(0, color='grey', linestyle='--')
axes[0].set_title('Climatic Distance')

# Plot 2: 'climatic distance' and 'genomi_offset'
sns.boxplot(data=combined_results[combined_results['dataset'].isin(['climatic_distance', 'genomic_offset'])], order=order_touse,
            y=variable, x='site', hue='dataset', ax=axes[1], palette=palette)
axes[1].axhline(0, color='grey', linestyle='--')
axes[1].set_title('Climatic Distance and Genomi Offset')

# Plot 3: 'climatic distance', 'genomi_offset', and 'climatic_dsitance model'
sns.boxplot(data=combined_results[combined_results['dataset'].isin(['climatic_distance', 'genomic_offset', 'climatic_distance_model'])], order=order_touse,
            y=variable, x='site', hue='dataset', ax=axes[2], palette=palette)
axes[2].axhline(0, color='grey', linestyle='--')
axes[2].set_title('Climatic Distance, Genomi Offset, and Climatic Distance Model')

# Plot 4: All datasets
sns.boxplot(data=combined_results[combined_results['dataset'].isin(['climatic_distance', 'genomic_offset', 'climatic_distance_model', 'baseline_pairwise_samples'])], 
            y=variable, x='site', hue='dataset', order=order_touse, ax=axes[3], palette=palette)
axes[3].axhline(0, color='grey', linestyle='--')


# Plot 4: All datasets
sns.boxplot(data=combined_results[combined_results['dataset'].isin(['baseline_pairwise_samples', 'genomic_offset'])], 
            y=variable, x='site', hue='dataset', order=order_touse, ax=axes[4], palette=palette)
axes[4].axhline(0, color='grey', linestyle='--')


# Plot 4: All datasets
sns.boxplot(data=combined_results[combined_results['dataset'].isin(['climatic_distance', 'climatic_distance_model'])], 
            y=variable, x='site', hue='dataset', order=order_touse, ax=axes[5], palette=palette)
axes[5].axhline(0, color='grey', linestyle='--')


# Set common properties
for ax in axes.flat:
    ax.grid(axis='y')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

# Adjust layout
plt.tight_layout()

# Display the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

variable = 'sp_correlation'
order_touse = sort_bio1_int

# Define a custom color palette mapping each dataset to a specific color
palette = {
    'climatic_distance': 'white',  # Light green
    'genomic_offset': '#32CD32',     # Dark green
    'climatic_distance_model': 'grey',  # Lime green
    'baseline_pairwise_samples': 'black' # Forest green
}


# Create 4 subplots (4 rows, 1 column)
fig, axes = plt.subplots(6, 1, figsize=(12, 26), sharex=True, sharey=True)

# Plot 1: Only 'climatic distance'
sns.boxplot(data=combined_results[combined_results['dataset'] == 'climatic_distance'], order=order_touse,
            y=variable, x='site', hue='dataset', ax=axes[0], palette=palette)
axes[0].axhline(0, color='grey', linestyle='--')
axes[0].set_title('Climatic Distance')

# Plot 2: 'climatic distance' and 'genomi_offset'
sns.boxplot(data=combined_results[combined_results['dataset'].isin(['climatic_distance', 'genomic_offset'])], order=order_touse,
            y=variable, x='site', hue='dataset', ax=axes[1], palette=palette)
axes[1].axhline(0, color='grey', linestyle='--')
axes[1].set_title('Climatic Distance and Genomi Offset')

# Plot 3: 'climatic distance', 'genomi_offset', and 'climatic_dsitance model'
sns.boxplot(data=combined_results[combined_results['dataset'].isin(['climatic_distance', 'genomic_offset', 'climatic_distance_model'])], order=order_touse,
            y=variable, x='site', hue='dataset', ax=axes[2], palette=palette)
axes[2].axhline(0, color='grey', linestyle='--')
axes[2].set_title('Climatic Distance, Genomi Offset, and Climatic Distance Model')

# Plot 4: All datasets
sns.boxplot(data=combined_results[combined_results['dataset'].isin(['climatic_distance', 'genomic_offset', 'climatic_distance_model', 'baseline_pairwise_samples'])], 
            y=variable, x='site', hue='dataset', order=order_touse, ax=axes[3], palette=palette)
axes[3].axhline(0, color='grey', linestyle='--')


# Plot 4: All datasets
sns.boxplot(data=combined_results[combined_results['dataset'].isin(['baseline_pairwise_samples', 'genomic_offset'])], 
            y=variable, x='site', hue='dataset', order=order_touse, ax=axes[4], palette=palette)
axes[4].axhline(0, color='grey', linestyle='--')


# Plot 4: All datasets
sns.boxplot(data=combined_results[combined_results['dataset'].isin(['climatic_distance', 'climatic_distance_model'])], 
            y=variable, x='site', hue='dataset', order=order_touse, ax=axes[5], palette=palette)
axes[5].axhline(0, color='grey', linestyle='--')


# Set common properties
for ax in axes.flat:
    ax.grid(axis='y')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

# Adjust layout
plt.tight_layout()

# Display the plot
plt.show()


In [ ]:
variable = 'sp_correlation'
order_touse = sort_bio1
s_value = 7
alpha_value = 0.6 
# Define a custom color palette mapping each dataset to a specific color
palette_stripplot = {
    'climatic_distance': 'darkgrey',  # Light green
    'genomic_offset': '#32CD32',     # Dark green
    'climatic_distance_model': 'grey',  # Lime green
    'baseline_pairwise_samples': 'black' # Forest green
}

palette_boxplot = {
    'climatic_distance': 'lightgrey',  # Light green
    'genomic_offset': '#32CD32',     # Dark green
    'climatic_distance_model': 'grey',  # Lime green
    'baseline_pairwise_samples': 'black' # Forest green

}

#combined_results.
# Create 4 subplots (4 rows, 1 column)
fig, axes = plt.subplots(2, 1, figsize=(15, 10), sharex=True, sharey=True)

# Plot 1: Only 'climatic distance'
sns.stripplot(data=combined_results[combined_results['dataset'] == 'climatic_distance'], order=order_touse,s=s_value,alpha=alpha_value,
            y=variable, x='site', hue='dataset', ax=axes[0], palette=palette_stripplot)

sns.boxplot(data=combined_results[combined_results['dataset'] == 'climatic_distance'], order = sort_bio1.astype(int),
            y=variable, x='site', hue='dataset', ax=axes[0],fill=False,
            showfliers=False, color='black',palette=palette_boxplot)

axes[0].axhline(0, color='grey', linestyle='--')
axes[0].set_title('Climatic Distance')

# Plot 2: 'climatic distance' and 'genomi_offset'
sns.stripplot(data=combined_results[combined_results['dataset'].isin(['climatic_distance', 'genomic_offset'])], order=order_touse,s=s_value,alpha=alpha_value,
            y=variable, x='site', hue='dataset', ax=axes[1], palette=palette_stripplot, dodge=True)

box_plot = sns.boxplot(data=combined_results[combined_results['dataset'].isin(['climatic_distance', 'genomic_offset'])], 
                       order=sort_bio1.astype(int), y=variable, x='site', 
                       hue='dataset', ax=axes[1],fill=False,
                       showfliers=False, palette=palette_boxplot)

# Create the boxplot
axes[1].axhline(0, color='grey', linestyle='--')
axes[1].set_title('Climatic Distance and Genomi Offset')

# Set common properties
for ax in axes.flat:
    ax.grid(axis='y')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)


axes[0].grid(True, which='major', axis='both', linestyle='--', linewidth=0.7)
axes[1].grid(True, which='major', axis='both', linestyle='--', linewidth=0.7)


# Adjust layout
plt.tight_layout()

# Display the plot
plt.show()


In [ ]:
order_touse = sort_bio1

In [ ]:
variable = 'sp_correlation'
order_touse = sort_bio1
s_value = 9
alpha_value = 0.6 
font_size = 12

# Define a custom color palette mapping each dataset to a specific color
green = '#2C5530'
grey = '#969C98'
# Define a custom color palette mapping each dataset to a specific color
palette_stripplot = {
    'climatic_distance': 'darkgrey',  # Light green
    'genomic_offset': green,     # Dark green
    'climatic_distance_model': grey,  # Lime green
    'baseline_pairwise_samples': 'black' # Forest green
}

palette_boxplot = {
    'climatic_distance': 'lightgrey',  # Light green
    'genomic_offset': green,     # Dark green
    'climatic_distance_model': grey,  # Lime green
    'baseline_pairwise_samples': 'black' # Forest green

}

# Calculate medians of 'sp_correlation' for 'results' only to determine the order
medians_results = results.groupby('site')[variable].mean().sort_values()
order_touse = medians_results.index
#combined_results.
# Create 4 subplots (4 rows, 1 column)
fig, ax = plt.subplots(figsize=(18, 6))

# Plot 2: 'climatic distance' and 'genomi_offset'
stripplot = sns.stripplot(data=combined_results[combined_results['dataset'].isin(['climatic_distance', 'genomic_offset'])], order=order_touse,s=s_value,alpha=alpha_value,
            y=variable, x='site', hue='dataset', palette=palette_stripplot, dodge=True)

box_plot = sns.boxplot(data=combined_results[combined_results['dataset'].isin(['climatic_distance', 'genomic_offset'])], 
                       order=sort_bio1.astype(int), y=variable, x='site', 
                       hue='dataset',fill=False,
                       showfliers=False, palette=palette_boxplot,
                      legend= False)


# Add stars for statistical significance and color them based on which one is higher
for site in test_results:
    p_value = test_results[site].pvalue
    if p_value < 0.05:
        x_position = list(order_touse.astype(int)).index(site)
        y_max = combined_results[combined_results['site'] == site][variable].max()
        site_data = merged_data[merged_data['site'] == site]
        genomic_mean = site_data[variable + '_geno'].mean()
        climatic_mean = site_data[variable + '_clim'].mean()
        
        if genomic_mean > climatic_mean:
            star_color = green  # Green if genomic offset is higher
        else:
            star_color = grey  # Grey if climatic distance is higher

        ymax = 0.95
        # Add stars for significance with the appropriate color
        ax.text(x_position, ymax + 0.05, "*", ha='center', va='bottom', color=star_color, fontsize=23)

        # Add horizontal bracket (bracket over the comparison)
        ax.annotate('', xy=(x_position,ymax + 0.06), xycoords='data',
                    xytext=(x_position, ymax - 0.06), textcoords='data',
                    arrowprops={'arrowstyle': ']-', 'color': 'grey', 'linewidth': 1.5})

ax.grid(axis='y')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)


ax.grid(True, which='major', axis='y', linestyle='--', linewidth=0.7)
# Customizing the legend to show only stripplot legend items
# Customizing the legend to show only stripplot legend items
handles, labels = ax.get_legend_handles_labels()
legend_handles = handles[:len(stripplot.get_legend_handles_labels()[0])]  # Keep only stripplot legend handles
legend_labels = ['Genomic Offset', 'Climatic Distance']  # Custom legend labels

# Adding the filtered legend back to the plot
ax.legend(legend_handles, legend_labels, bbox_to_anchor=(1.01, 0.97), loc='upper left', borderaxespad=0.)
ax.set_ylim(-0.5, 1.1)  

ax.set_xlabel('Experimental Sites')
ax.set_ylabel('Spearman Correlation')
# Adjust layout
plt.tight_layout()

plt.rc('font', family='sans-serif', size=font_size, weight='normal')  # Regular font size
plt.rc('axes', titlesize=font_size, labelsize=font_size)  # Title and label size for axes
plt.rc('xtick', labelsize=font_size)  # X tick labels size
plt.rc('ytick', labelsize=font_size)  # Y tick labels size
dark_grey = '#4D4D4D' 
plt.tick_params(axis='both', colors=dark_grey)  # Set tick colors

plt.savefig('spearman_go_vs_climatic_distance_ordergo.png')
plt.savefig('spearman_go_vs_climatic_distance_ordergo.svg')

# Display the plot
plt.show()


In [ ]:
from itertools import combinations
from scipy.stats import mannwhitneyu
import pandas as pd

# Define the order of your groups
groups = ['climatic_distance',  'climatic_distance_model', 'genomic_offset', 'baseline_pairwise_samples']  # ['climatic_distance',  'climatic_distance_model', 'genomic_offset', 'baseline_pairwise_samples']

# Create an empty DataFrame to store pairwise p-values
p_values = pd.DataFrame(index=groups, columns=groups)

# Loop over each pair of groups and perform Wilcoxon rank-sum test
for group1, group2 in combinations(groups, 2):
    group1_data = combined_results[combined_results['dataset'] == group1]['sp_correlation']
    group2_data = combined_results[combined_results['dataset'] == group2]['sp_correlation']
    
    # Perform the Wilcoxon rank-sum test
    stat, p_value = mannwhitneyu(group1_data, group2_data, alternative='two-sided')
    
    # Store the p-value
    p_values.loc[group1, group2] = p_value
    p_values.loc[group2, group1] = p_value  # fill in symmetric value

In [ ]:
p_values

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import mannwhitneyu

# Define the order of your groups
order = ['climatic_distance', 'climatic_distance_model', 'genomic_offset', 'baseline_pairwise_samples']
ticks = ['Climatic\nDistance', 'Climatic\nDistance Model', 'Genomic\nOffset', 'Within Site\nSamples']

# Initialize the plot
plt.figure(figsize=(8, 8)) 
ax = sns.stripplot(data=combined_results, y='sp_correlation', x='dataset', order=order, color='grey', alpha=0.3, s=7.5, zorder=1)
sns.boxplot(data=combined_results, y='sp_correlation', x='dataset', order=order, showfliers=False, color='black', width=0.5, zorder=2, fill=False)

# Perform pairwise Mann-Whitney U tests between the relevant groups
group_genomic_offset = combined_results[combined_results['dataset'] == 'genomic_offset']['sp_correlation']
group_climatic_distance = combined_results[combined_results['dataset'] == 'climatic_distance']['sp_correlation']
group_climatic_distance_model = combined_results[combined_results['dataset'] == 'climatic_distance_model']['sp_correlation']

# Mann-Whitney U test between 'genomic_offset' and 'climatic_distance'
stat, p_value_1 = mannwhitneyu(group_genomic_offset, group_climatic_distance, alternative='two-sided')

# Mann-Whitney U test between 'genomic_offset' and 'climatic_distance_model'
stat, p_value_2 = mannwhitneyu(group_genomic_offset, group_climatic_distance_model, alternative='two-sided')

# Define significance threshold
significance_threshold = 0.05

# Add stars for significant comparisons
ymax = combined_results['sp_correlation'].max() + 0.1
height_increment = 0.1

# Add stars and brackets if significant for 'genomic_offset' vs 'climatic_distance'
if p_value_1 < significance_threshold:
    x1, x2 = order.index('genomic_offset'), order.index('climatic_distance')  # positions of the groups
    y, h, col = ymax, height_increment, 'grey'  # height and color of the annotation
    ax.plot([x1, x1, x2, x2], [y, y + h, y + h, y], lw=1.5, c=col)
    ax.text((x1 + x2) * 0.5, y + h, "*", ha='center', va='bottom', color='grey', fontsize=23)

# Add stars and brackets if significant for 'genomic_offset' vs 'climatic_distance_model'
if p_value_2 < significance_threshold:
    x1, x2 = order.index('genomic_offset'), order.index('climatic_distance_model')  # positions of the groups
    y, h, col = ymax + 0.3, height_increment, 'grey'  # increment height slightly to avoid overlap
    ax.plot([x1, x1, x2, x2], [y, y + h, y + h, y], lw=1.5, c=col)
    ax.text((x1 + x2) * 0.5, y + h, "*", ha='center', va='bottom', color='grey', fontsize=23)

# Customize axis labels and ticks
plt.xticks([0, 1, 2, 3], ticks)
plt.xlabel('')
plt.ylabel('Spearman Correlation')

# Add grid to the background for y-axis
plt.grid(True, which='major', axis='y', linestyle='--', linewidth=0.7)

# Remove all spines
sns.despine(left=True, right=True, top=True, bottom=True)


plt.savefig('go_cm_pairwise_samples_overall.png', )
plt.savefig('go_cm_pairwise_samples_overall.svg')

# Show the plot
plt.show()


In [ ]:
variable = 'sp_correlation'
order_touse = sort_bio1
s_value = 9
alpha_value = 0.6 

green = '#2C5530'
grey = '#969C98'
# Define a custom color palette mapping each dataset to a specific color
palette_stripplot = {
    'climatic_distance': 'darkgrey',  # Light green
    'genomic_offset': green,     # Dark green
    'climatic_distance_model': grey,  # Lime green
    'baseline_pairwise_samples': 'black' # Forest green
}

palette_boxplot = {
    'climatic_distance': 'lightgrey',  # Light green
    'genomic_offset': green,     # Dark green
    'climatic_distance_model': grey,  # Lime green
    'baseline_pairwise_samples': 'black' # Forest green

}

# Calculate medians of 'sp_correlation' for 'results' only to determine the order
medians_results = results.groupby('site')[variable].mean().sort_values()
#combined_results.
# Create 4 subplots (4 rows, 1 column)
fig, ax = plt.subplots(figsize=(18, 6))

# Plot 2: 'climatic distance' and 'genomi_offset'
stripplot = sns.stripplot(data=combined_results[combined_results['dataset'].isin(['climatic_distance', 'genomic_offset'])], order=order_touse,s=s_value,alpha=alpha_value,
            y=variable, x='site', hue='dataset', palette=palette_stripplot, dodge=True)

box_plot = sns.boxplot(data=combined_results[combined_results['dataset'].isin(['climatic_distance', 'genomic_offset'])], 
                       order=sort_bio1.astype(int), y=variable, x='site', 
                       hue='dataset',fill=False,
                       showfliers=False, palette=palette_boxplot,
                      legend= False)


# Add stars for statistical significance and color them based on which one is higher
for site in test_results:
    p_value = test_results[site].pvalue
    if p_value < 0.05:
        x_position = list(order_touse.astype(int)).index(site)
        y_max = combined_results[combined_results['site'] == site][variable].max()
        site_data = merged_data[merged_data['site'] == site]
        genomic_mean = site_data[variable + '_geno'].mean()
        climatic_mean = site_data[variable + '_clim'].mean()
        
        if genomic_mean > climatic_mean:
            star_color = green  # Green if genomic offset is higher
        else:
            star_color = grey  # Grey if climatic distance is higher

        ymax = 0.95
        # Add stars for significance with the appropriate color
        ax.text(x_position, ymax + 0.05, "*", ha='center', va='bottom', color=star_color, fontsize=23)

        # Add horizontal bracket (bracket over the comparison)
        ax.annotate('', xy=(x_position,ymax + 0.06), xycoords='data',
                    xytext=(x_position, ymax - 0.06), textcoords='data',
                    arrowprops={'arrowstyle': ']-', 'color': 'grey', 'linewidth': 1.5})

ax.grid(axis='y')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)


ax.grid(True, which='major', axis='y', linestyle='--', linewidth=0.7)
# Customizing the legend to show only stripplot legend items
# Customizing the legend to show only stripplot legend items
handles, labels = ax.get_legend_handles_labels()
legend_handles = handles[:len(stripplot.get_legend_handles_labels()[0])]  # Keep only stripplot legend handles
legend_labels = ['Genomic Offset', 'Climatic Distance']  # Custom legend labels

# Adding the filtered legend back to the plot
ax.legend(legend_handles, legend_labels, bbox_to_anchor=(1.01, 0.97), loc='upper left', borderaxespad=0.)
ax.set_ylim(-0.5, 1.1)  

ax.set_xlabel('Experimental Sites')
ax.set_ylabel('Spearman Correlation')
# Adjust layout


plt.rc('font', family='sans-serif', size=font_size, weight='normal')  # Regular font size
plt.rc('axes', titlesize=font_size, labelsize=font_size)  # Title and label size for axes
plt.rc('xtick', labelsize=font_size)  # X tick labels size
plt.rc('ytick', labelsize=font_size)  # Y tick labels size
dark_grey = '#4D4D4D' 
plt.tick_params(axis='both', colors=dark_grey)  # Set tick colors

plt.savefig('spearman_go_vs_climatic_distance_orderbio1.png')
plt.savefig('spearman_go_vs_climatic_distance_orderbio1.svg')


plt.tight_layout()

# Display the plot
plt.show()


In [ ]:
variable = 'sp_correlation'
order_touse = sort_bio1
s_value = 9
alpha_value = 0.5

green = '#2C5530'
grey = '#969C98'
black = '#485696'
# Define a custom color palette mapping each dataset to a specific color
palette_stripplot = {
    'climatic_distance': 'darkgrey',  # Light green
    'genomic_offset': green,     # Dark green
    'climatic_distance_model': grey,  # Lime green
    'baseline_pairwise_samples': black # Forest green
}

palette_boxplot = {
    'climatic_distance': 'lightgrey',  # Light green
    'genomic_offset': green,     # Dark green
    'climatic_distance_model': grey,  # Lime green
    'baseline_pairwise_samples': black # Forest green

}

# Calculate medians of 'sp_correlation' for 'results' only to determine the order
medians_results = results.groupby('site')[variable].mean().sort_values()
#combined_results.
# Create 4 subplots (4 rows, 1 column)
fig, ax = plt.subplots(figsize=(18, 6))

# Plot 2: 'climatic distance' and 'genomi_offset'
stripplot = sns.stripplot(data=combined_results[combined_results['dataset'].isin(['climatic_distance', 'genomic_offset', 'baseline_pairwise_samples'])], order=order_touse,s=s_value,alpha=alpha_value,
            y=variable, x='site', hue='dataset', palette=palette_stripplot, dodge=True, zorder= 1)

box_plot = sns.boxplot(data=combined_results[combined_results['dataset'].isin(['climatic_distance', 'genomic_offset', 'baseline_pairwise_samples'])], 
                       order=sort_bio1.astype(int), y=variable, x='site', 
                       hue='dataset',fill=False,
                       showfliers=False, palette=palette_boxplot,
                      legend= False,
                      zorder= 2)



ax.grid(axis='y')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)


ax.grid(True, which='major', axis='y', linestyle='--', linewidth=0.7)
# Customizing the legend to show only stripplot legend items
# Customizing the legend to show only stripplot legend items
handles, labels = ax.get_legend_handles_labels()
legend_handles = handles[:len(stripplot.get_legend_handles_labels()[0])]  # Keep only stripplot legend handles
legend_labels = ['Genomic Offset', 'Climatic Distance']  # Custom legend labels

# Adding the filtered legend back to the plot
ax.legend(legend_handles, legend_labels, bbox_to_anchor=(1.01, 0.97), loc='upper left', borderaxespad=0.)
ax.set_ylim(-0.5, 1.1)  

ax.set_xlabel('Experimental Sites')
ax.set_ylabel('Spearman Correlation')
# Adjust layout
plt.tight_layout()

# Display the plot
plt.show()


In [ ]:
variable = 'kendall_tau'
order_touse = sort_bio1
s_value = 7
alpha_value = 0.6 
# Define a custom color palette mapping each dataset to a specific color
palette_stripplot = {
    'climatic_distance': 'darkgrey',  # Light green
    'genomic_offset': 'green',     # Dark green
    'climatic_distance_model': 'grey',  # Lime green
    'baseline_pairwise_samples': 'black' # Forest green
}

palette_boxplot = {
    'climatic_distance': 'lightgrey',  # Light green
    'genomic_offset': '#32CD32',     # Dark green
    'climatic_distance_model': 'grey',  # Lime green
    'baseline_pairwise_samples': 'black' # Forest green

}


# Calculate medians of 'sp_correlation' for 'results' only to determine the order
medians_results = results.groupby('site')[variable].mean().sort_values()

#combined_results.
# Create 4 subplots (4 rows, 1 column)
fig, ax = plt.subplots(figsize=(18, 6))

# Plot 2: 'climatic distance' and 'genomi_offset'
stripplot = sns.stripplot(data=combined_results[combined_results['dataset'].isin(['climatic_distance', 'genomic_offset'])], order=order_touse,s=s_value,alpha=alpha_value,
            y=variable, x='site', hue='dataset', palette=palette_stripplot, dodge=True)

box_plot = sns.boxplot(data=combined_results[combined_results['dataset'].isin(['climatic_distance', 'genomic_offset'])], 
                       order=sort_bio1.astype(int), y=variable, x='site', 
                       hue='dataset',fill=False,
                       showfliers=False, palette=palette_boxplot,
                      legend= False)


# Add stars for statistical significance and color them based on which one is higher
for site in test_results:
    p_value = test_results[site].pvalue
    if p_value < 0.05:
        x_position = list(order_touse.astype(int)).index(site)
        y_max = combined_results[combined_results['site'] == site][variable].max()
        site_data = merged_data[merged_data['site'] == site]
        genomic_mean = site_data[variable + '_geno'].mean()
        climatic_mean = site_data[variable + '_clim'].mean()
        
        if genomic_mean > climatic_mean:
            star_color = 'green'  # Green if genomic offset is higher
        else:
            star_color = 'darkgrey'  # Grey if climatic distance is higher

        ymax = 0.7
        # Add stars for significance with the appropriate color
        ax.text(x_position, ymax + 0.05, "*", ha='center', va='bottom', color=star_color, fontsize=16)

        # Add horizontal bracket (bracket over the comparison)
        ax.annotate('', xy=(x_position,ymax + 0.06), xycoords='data',
                    xytext=(x_position, ymax - 0.06), textcoords='data',
                    arrowprops={'arrowstyle': ']-', 'color': 'grey', 'linewidth': 1.5})

ax.grid(axis='y')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)


ax.grid(True, which='major', axis='y', linestyle='--', linewidth=0.7)
# Customizing the legend to show only stripplot legend items
# Customizing the legend to show only stripplot legend items
handles, labels = ax.get_legend_handles_labels()
legend_handles = handles[:len(stripplot.get_legend_handles_labels()[0])]  # Keep only stripplot legend handles
legend_labels = ['Genomic Offset', 'Climatic Distance']  # Custom legend labels

# Adding the filtered legend back to the plot
ax.legend(legend_handles, legend_labels, bbox_to_anchor=(1.01, 0.97), loc='upper left', borderaxespad=0.)
ax.set_ylim(-0.4, 0.9)  

ax.set_xlabel('Experimental Sites')
ax.set_ylabel('Spearman Correlation')
# Adjust layout
plt.tight_layout()

# Display the plot
plt.show()


In [ ]:
variable = 'top_30'
from scipy.stats import ttest_rel, wilcoxon

# To store the p-values
test_results = {}

for site in combined_results['site'].unique():
    
    site_data = combined_results[combined_results['site'] == site]    
    clim = site_data[site_data['dataset'] == 'climatic_distance'][variable] 
    geno = site_data[site_data['dataset'] == 'genomic_offset'][variable]
    # If not normally distributed, use Wilcoxon signed-rank test
    test_results[site] = wilcoxon(geno, clim)

In [ ]:
combined_results

In [ ]:



  # This will give you the p-values for each sit

In [ ]:
clim['sp_correlation']

In [ ]:
variable = 'sp_correlation'
order_touse = sort_bio1
s_value = 7
alpha_value = 0.6 
# Define a custom color palette mapping each dataset to a specific color
palette = {
    'climatic_distance': 'lightgrey',  # Light green
    'genomic_offset': '#32CD32',     # Dark green
    'climatic_distance_model': 'grey',  # Lime green
    'baseline_pairwise_samples': 'black' # Forest green
}

#combined_results.
# Create 4 subplots (4 rows, 1 column)
fig, axes = plt.subplots(6, 1, figsize=(12, 26), sharex=True, sharey=True)

# Plot 1: Only 'climatic distance'
sns.stripplot(data=combined_results[combined_results['dataset'] == 'climatic_distance'], order=order_touse,s=s_value,alpha=alpha_value,
            y=variable, x='site', hue='dataset', ax=axes[0], palette=palette)
axes[0].axhline(0, color='grey', linestyle='--')
axes[0].set_title('Climatic Distance')

# Plot 2: 'climatic distance' and 'genomi_offset'
sns.stripplot(data=combined_results[combined_results['dataset'].isin(['climatic_distance', 'genomic_offset'])], order=order_touse,s=s_value,alpha=alpha_value,
            y=variable, x='site', hue='dataset', ax=axes[1], palette=palette)
axes[1].axhline(0, color='grey', linestyle='--')
axes[1].set_title('Climatic Distance and Genomi Offset')

# Plot 3: 'climatic distance', 'genomi_offset', and 'climatic_dsitance model'
sns.stripplot(data=combined_results[combined_results['dataset'].isin(['climatic_distance', 'genomic_offset', 'climatic_distance_model'])], order=order_touse,s=s_value,alpha=alpha_value,
            y=variable, x='site', hue='dataset', ax=axes[2], palette=palette)
axes[2].axhline(0, color='grey', linestyle='--')
axes[2].set_title('Climatic Distance, Genomi Offset, and Climatic Distance Model')

# Plot 4: All datasets
sns.stripplot(data=combined_results[combined_results['dataset'].isin(['climatic_distance', 'genomic_offset', 'climatic_distance_model', 'baseline_pairwise_samples'])],s=s_value,alpha=alpha_value,
            y=variable, x='site', hue='dataset', order=order_touse, ax=axes[3], palette=palette)
axes[3].axhline(0, color='grey', linestyle='--')


# Plot 4: All datasets
sns.stripplot(data=combined_results[combined_results['dataset'].isin(['baseline_pairwise_samples', 'genomic_offset'])],s=s_value,alpha=alpha_value,
            y=variable, x='site', hue='dataset', order=order_touse, ax=axes[4], palette=palette)
axes[4].axhline(0, color='grey', linestyle='--')


# Plot 4: All datasets
sns.stripplot(data=combined_results[combined_results['dataset'].isin(['climatic_distance', 'climatic_distance_model'])],s=s_value,alpha=alpha_value,
            y=variable, x='site', hue='dataset', order=order_touse, ax=axes[5], palette=palette)
axes[5].axhline(0, color='grey', linestyle='--')


# Set common properties
for ax in axes.flat:
    ax.grid(axis='y')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

# Adjust layout
plt.tight_layout()

# Display the plot
plt.show()


In [ ]:
# Calculate medians of 'sp_correlation' for 'results' only to determine the order
medians_results = cd_results.groupby('site')['sp_correlation'].mean().sort_values()

# Set the size of the plot
plt.figure(figsize=(12, 6))

# Create the violin plot, using 'hue' to distinguish between 'results' and 'results_cd'
sns.boxplot(data=combined_results, y='sp_correlation', x='site', hue='dataset', order=medians_results.index)

# Draw a horizontal line at y=0
plt.axhline(0, color='grey', linestyle='--')

# Grid and spine settings
plt.grid(axis='y')
ax = plt.gca()  # Get current axes
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add labels and title
plt.xlabel('Site')

# Add labels and title
plt.title('Top 10 ecotypes')

# Display the plot
plt.show()

In [ ]:
# Calculate medians of 'sp_correlation' for 'results' only to determine the order
medians_results = cd_results.groupby('site')['kendall_tau'].mean().sort_values()

# Set the size of the plot
plt.figure(figsize=(12, 6))

# Create the violin plot, using 'hue' to distinguish between 'results' and 'results_cd'
sns.stripplot(data=combined_results, y='kendall_tau', x='site', hue='dataset', order=medians_results.index.astype(str), s=8, alpha = 0.5)

# Draw a horizontal line at y=0
plt.axhline(0, color='grey', linestyle='--')

# Grid and spine settings
plt.grid(axis='y')
ax = plt.gca()  # Get current axes
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add labels and title
plt.xlabel('Site')

# Display the plot
plt.show()

In [ ]:
combined_results

In [ ]:
# Calculate medians of 'sp_correlation' for 'results' only to determine the order
medians_results = cd_results.groupby('site')['r_squared'].mean().sort_values()

# Set the size of the plot
plt.figure(figsize=(12, 6))

# Create the violin plot, using 'hue' to distinguish between 'results' and 'results_cd'
sns.boxplot(data=combined_results[combined_results['dataset']!= 'baseline_pairwise_samples'], y='r_squared', x='site', hue='dataset', order=medians_results.index)

# Draw a horizontal line at y=0
plt.axhline(0, color='grey', linestyle='--')

# Grid and spine settings
plt.grid(axis='y')
ax = plt.gca()  # Get current axes
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add labels and title
plt.xlabel('Site')

# Add labels and title
plt.title('Top 10 ecotypes')

# Display the plot
plt.show()

In [ ]:
# Calculate medians of 'sp_correlation' for 'results' only to determine the order
medians_results = cd_results.groupby('site')['pearsonr'].mean().sort_values()

# Set the size of the plot
plt.figure(figsize=(12, 6))

# Create the violin plot, using 'hue' to distinguish between 'results' and 'results_cd'
sns.boxplot(data=combined_results[combined_results['dataset']!= 'baseline_pairwise_samples'], y='pearsonr', x='site', hue='dataset', order=medians_results.index)

# Draw a horizontal line at y=0
plt.axhline(0, color='grey', linestyle='--')

# Grid and spine settings
plt.grid(axis='y')
ax = plt.gca()  # Get current axes
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add labels and title
plt.xlabel('Site')

# Add labels and title
plt.title('Top 10 ecotypes')

# Display the plot
plt.show()

In [ ]:
# Calculate medians of 'sp_correlation' for 'results' only to determine the order
medians_results = cd_results.groupby('site')['sp_correlation'].mean().sort_values()

# Set the size of the plot
plt.figure(figsize=(12, 6))

# Create the violin plot, using 'hue' to distinguish between 'results' and 'results_cd'
sns.boxplot(data=combined_results, y='sp_correlation', x='site', hue='dataset', order=sort_bio1)

# Draw a horizontal line at y=0
plt.axhline(0, color='grey', linestyle='--')

# Grid and spine settings
plt.grid(axis='y')
ax = plt.gca()  # Get current axes
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add labels and title
plt.xlabel('Site')

# Add labels and title
plt.title('Top 10 ecotypes')

# Display the plot
plt.show()

In [ ]:
# Calculate medians of 'sp_correlation' for 'results' only to determine the order
medians_results = cd_results.groupby('site')['sp_correlation'].mean().sort_values()

# Set the size of the plot
plt.figure(figsize=(12, 6))

# Create the violin plot, using 'hue' to distinguish between 'results' and 'results_cd'
sns.boxplot(data=combined_results, y='sp_correlation', x='site', hue='dataset', order=medians_results.index.astype(str))

# Draw a horizontal line at y=0
plt.axhline(0, color='grey', linestyle='--')

# Grid and spine settings
plt.grid(axis='y')
ax = plt.gca()  # Get current axes
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add labels and title
plt.xlabel('Site')

# Add labels and title
plt.title('Top 10 ecotypes')

# Display the plot
plt.show()

In [ ]:
# Calculate medians of 'sp_correlation' for 'results' only to determine the order
medians_results = cd_results.groupby('site')['sp_correlation'].mean().sort_values()

# Set the size of the plot
plt.figure(figsize=(12, 6))

# Create the violin plot, using 'hue' to distinguish between 'results' and 'results_cd'
sns.stripplot(data=combined_results, y='sp_correlation', x='site', hue='dataset', order=medians_results.index.astype(str), s=8, alpha = 0.5)

# Draw a horizontal line at y=0
plt.axhline(0, color='grey', linestyle='--')

# Grid and spine settings
plt.grid(axis='y')
ax = plt.gca()  # Get current axes
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add labels and title
plt.xlabel('Site')

# Display the plot
plt.show()

In [ ]:
# Calculate medians of 'sp_correlation' for 'results' only to determine the order
medians_results = results.groupby('site')['top_20'].mean().sort_values()

# Set the size of the plot
plt.figure(figsize=(12, 6))

# Create the violin plot, using 'hue' to distinguish between 'results' and 'results_cd'
sns.boxplot(data=combined_results, y='top_20', x='site', hue='dataset', order=medians_results.index)

# Draw a horizontal line at y=0
plt.axhline(0, color='grey', linestyle='--')

# Grid and spine settings
plt.grid(axis='y')
ax = plt.gca()  # Get current axes
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add labels and title
plt.xlabel('Site')

# Add labels and title
plt.title('Top 10 ecotypes')

# Display the plot
plt.show()

In [ ]:
# Calculate medians of 'sp_correlation' for 'results' only to determine the order
medians_results = cd_results.groupby('site')['top_30'].mean().sort_values()

# Set the size of the plot
plt.figure(figsize=(12, 6))

# Create the violin plot, using 'hue' to distinguish between 'results' and 'results_cd'
sns.boxplot(data=combined_results, y='top_30', x='site', hue='dataset', order=medians_results.index)

# Draw a horizontal line at y=0
plt.axhline(0, color='grey', linestyle='--')

# Grid and spine settings
plt.grid(axis='y')
ax = plt.gca()  # Get current axes
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add labels and title
plt.xlabel('Site')
plt.title('Top 20 ecotypes')

# Display the plot
plt.show()

In [ ]:
# Calculate medians of 'sp_correlation' for 'results' only to determine the order
medians_results = cd_results.groupby('site')['top_50'].mean().sort_values()

# Set the size of the plot
plt.figure(figsize=(12, 6))

# Create the violin plot, using 'hue' to distinguish between 'results' and 'results_cd'
sns.boxplot(data=combined_results, y='top_50', x='site', hue='dataset', order=medians_results.index)

# Draw a horizontal line at y=0
plt.axhline(0, color='grey', linestyle='--')

# Grid and spine settings
plt.grid(axis='y')
ax = plt.gca()  # Get current axes
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add labels and title
plt.xlabel('Site')
plt.title('Top 20 ecotypes')


# Display the plot
plt.show()

In [ ]:
medians_results.index.astype(str)

In [ ]:
# Calculate medians of 'sp_correlation' for 'results' only to determine the order
medians_results = cd_results.groupby('site')['top_50'].mean().sort_values()

# Set the size of the plot
plt.figure(figsize=(12, 6))

# Create the violin plot, using 'hue' to distinguish between 'results' and 'results_cd'
sns.boxplot(data=combined_results, y='top_50', x='site', hue='dataset', order=medians_results.index.astype(str))

# Draw a horizontal line at y=0
plt.axhline(0, color='grey', linestyle='--')

# Grid and spine settings
plt.grid(axis='y')
ax = plt.gca()  # Get current axes
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add labels and title
plt.xlabel('Site')

# Display the plot
plt.show()

In [ ]:
medians_results = cd_results.groupby('site')['top_50'].mean().sort_values()

In [ ]:
order_str = [str(i) for i in medians_results.index]

In [ ]:
combined_results['site'] = combined_results['site'].astype(str)

In [ ]:
# Define the order as string for the 'site' column
order_str = ['12', '11', '9', '10', '28', '49', '57', '13', '23', '25', '53', '24', '5', '42', '46', '2', 
             '52', '54', '48', '1', '27', '55', '26', '37', '60', '32', '6', '43', '45', '4']



In [ ]:
# Set the size of the plot
plt.figure(figsize=(25, 6))

# Create the violin plot, using 'hue' to distinguish between 'results' and 'results_cd'
sns.stripplot(data=combined_results, y='top_50', x='site', hue='dataset', alpha = .5, order=order_str)

# Draw a horizontal line at y=0
plt.axhline(0, color='grey', linestyle='--')

# Grid and spine settings
plt.grid(axis='y')
ax = plt.gca()  # Get current axes
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add labels and title
plt.xlabel('Site')

# Display the plot
plt.show()

In [ ]:
# Set the size of the plot
plt.figure(figsize=(25, 6))

# Create the violin plot, using 'hue' to distinguish between 'results' and 'results_cd'
sns.stripplot(data=combined_results, y='top_50', x='site', hue='dataset', alpha = .5, order=sort_bio1, s= 7)

# Draw a horizontal line at y=0
plt.axhline(0, color='grey', linestyle='--')

# Grid and spine settings
plt.grid(axis='y')
ax = plt.gca()  # Get current axes
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add labels and title
plt.xlabel('Site')

# Display the plot
plt.show()

In [ ]:
# Set the size of the plot
plt.figure(figsize=(25, 6))

# Create the violin plot, using 'hue' to distinguish between 'results' and 'results_cd'
sns.swarmplot(data=combined_results, y='top_50', x='site', hue='dataset', alpha = .5, order=order_str)

# Draw a horizontal line at y=0
plt.axhline(0, color='grey', linestyle='--')

# Grid and spine settings
plt.grid(axis='y')
ax = plt.gca()  # Get current axes
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add labels and title
plt.xlabel('Site')

# Display the plot
plt.show()

In [ ]:
combined_results_new = pd.read_csv('chat.csv')

# Convert the 'site' column to string and apply the order again for the new dataset
combined_results_new['site'] = combined_results_new['site'].astype(str)

# Define the order as string for the 'site' column
order_str = ['12', '11', '9', '10', '28', '49', '57', '13', '23', '25', '53', '24', '5', '42', '46', '2', 
             '52', '54', '48', '1', '27', '55', '26', '37', '60', '32', '6', '43', '45', '4']

# Create the swarm plot
plt.figure(figsize=(12, 6))
sns.swarmplot(data=combined_results_new, y='top_50', x='site', hue='dataset', alpha=0.5, order=order_str)

# Draw a horizontal line at y=0
plt.axhline(0, color='grey', linestyle='--')

# Grid and spine settings
plt.grid(axis='y')
ax = plt.gca()  # Get current axes
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add labels and title
plt.xlabel('Site')
plt.title('Top 50 Ecotypes by Site')

# Display the plot
plt.show()

In [ ]:
results.columns

In [ ]:
results['site'] = results['site'].astype(int)

In [ ]:
combined_results.groupby('dataset')['sp_correlation'].mean()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Add a new column to differentiate between the two DataFrames
results['dataset'] = 'results'
cd_results['dataset'] = 'results_cd'

# Concatenate the two DataFrames
combined_results = pd.concat([results, cd_results])

# Calculate medians of 'sp_correlation' for 'results' only to determine the order
medians_results = cd_results.groupby('site')['r_squared'].mean().sort_values()

# Set the size of the plot
plt.figure(figsize=(12, 6))

# Create the violin plot, using 'hue' to distinguish between 'results' and 'results_cd'
sns.violinplot(data=combined_results, y='r_squared', x='site', hue='dataset', order=medians_results.index)

# Draw a horizontal line at y=0
plt.axhline(0, color='grey', linestyle='--')

# Grid and spine settings
plt.grid(axis='y')
ax = plt.gca()  # Get current axes
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add labels and title
plt.xlabel('Site')

# Display the plot
plt.show()


In [ ]:
combined_results.groupby('dataset')['sp_correlation'].mean()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Add a new column to differentiate between the two DataFrames
results['dataset'] = 'results'
cd_results['dataset'] = 'results_cd'

# Concatenate the two DataFrames
combined_results = pd.concat([results, cd_results])

# Calculate medians of 'sp_correlation' for 'results' only to determine the order
medians_results = cd_results.groupby('site')['sp_correlation'].median().sort_values()

# Set the size of the plot
plt.figure(figsize=(27, 6))

# Create the violin plot, using 'hue' to distinguish between 'results' and 'results_cd'
sns.violinplot(data=combined_results, y='sp_correlation', x='site', hue='dataset', order=medians_results.index)

# Draw a horizontal line at y=0
plt.axhline(0, color='grey', linestyle='--')

# Grid and spine settings
plt.grid(axis='y')
ax = plt.gca()  # Get current axes
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add labels and title
plt.xlabel('Site')

# Display the plot
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Add a new column to differentiate between the two DataFrames
results['dataset'] = 'results'
cd_results['dataset'] = 'results_cd'

# Concatenate the two DataFrames
combined_results = pd.concat([results, cd_results])

# Calculate medians of 'sp_correlation' for 'results' only to determine the order
medians_results = cd_results.groupby('site')['sp_correlation'].median().sort_values()

# Set the size of the plot
plt.figure(figsize=(12, 6))

# Create the violin plot, using 'hue' to distinguish between 'results' and 'results_cd'
sns.violinplot(data=combined_results, y='sp_correlation', x='site', hue='dataset', order=medians_results.index)

# Draw a horizontal line at y=0
plt.axhline(0, color='grey', linestyle='--')

# Grid and spine settings
plt.grid(axis='y')
ax = plt.gca()  # Get current axes
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add labels and title
plt.xlabel('Site')

# Display the plot
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Add a new column to differentiate between the two DataFrames
results['dataset'] = 'results'
cd_results['dataset'] = 'results_cd'

# Concatenate the two DataFrames
combined_results = pd.concat([results, cd_results])

# Calculate medians of 'sp_correlation' for 'results' only to determine the order
medians_results = cd_results.groupby('site')['sp_correlation'].median().sort_values()

# Set the size of the plot
plt.figure(figsize=(12, 6))

# Create the violin plot, using 'hue' to distinguish between 'results' and 'results_cd'
sns.boxplot(data=combined_results, y='sp_correlation', x='site', hue='dataset', order=medians_results.index)

# Draw a horizontal line at y=0
plt.axhline(0, color='grey', linestyle='--')

# Grid and spine settings
plt.grid(axis='y')
ax = plt.gca()  # Get current axes
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add labels and title
plt.xlabel('Site')

# Display the plot
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Add a new column to differentiate between the two DataFrames
results['dataset'] = 'results'
cd_results['dataset'] = 'results_cd'

# Concatenate the two DataFrames
combined_results = pd.concat([results, cd_results])

# Calculate medians of 'sp_correlation' for 'results' only to determine the order
medians_results = results.groupby('site')['sp_correlation'].median().sort_values()

# Set the size of the plot
plt.figure(figsize=(12, 6))

# Create the violin plot, using 'hue' to distinguish between 'results' and 'results_cd'
sns.boxplot(data=combined_results, y='sp_correlation', x='site', hue='dataset', order=medians_results.index)

# Draw a horizontal line at y=0
plt.axhline(0, color='grey', linestyle='--')

# Grid and spine settings
plt.grid(axis='y')
ax = plt.gca()  # Get current axes
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add labels and title
plt.title('sp_correlation')
plt.xlabel('Site')
plt.ylabel('sp_correlation')

# Display the plot
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Add a new column to differentiate between the two DataFrames
results['dataset'] = 'results'
cd_results['dataset'] = 'results_cd'

# Concatenate the two DataFrames
combined_results = pd.concat([results, cd_results])

# Calculate medians of 'sp_correlation' for 'results' only to determine the order
medians_results = cd_results.groupby('site')['sp_correlation'].median().sort_values()

# Set the size of the plot
plt.figure(figsize=(16, 6))

# Create the violin plot, using 'hue' to distinguish between 'results' and 'results_cd'
sns.swarmplot(data=combined_results, y='sp_correlation', x='site', hue='dataset', order=medians_results.index)

# Draw a horizontal line at y=0
plt.axhline(0, color='grey', linestyle='--')

# Grid and spine settings
plt.grid(axis='y')
ax = plt.gca()  # Get current axes
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add labels and title
plt.title('sp_correlation')
plt.xlabel('Site')
plt.ylabel('sp_correlation')

# Display the plot
plt.show()


In [ ]:
conc = pd.concat([ef_f, cd],axis=1)

In [ ]:
conc

In [ ]:
import os
import pickle

In [ ]:
files = os.listdir('../baypass_lastgen/individual_gfiles_last_gen/')

partitions = [int(file.split('_')[1].replace('.txt', '')) for file in files if '.txt' in file]

In [ ]:
partitions_r = {}
for i in range(len(partitions)):
    pickle_file_path = f'../baypass_lastgen/individual_gfiles_last_gen/loci_partition_{i}'
    with open(pickle_file_path, 'rb') as file:
        loci_f = pickle.load(file)
    results = pd.read_csv(f'results/partition{i}.csv')
    results['snp_id'] = loci_f
    partitions_r[i] = results
results = pd.concat(partitions_r).reset_index(drop=True)
#results.to_csv(f'results/split_{split}/binomial_reg_results_last_gen.csv',index=None)

In [ ]:
snp_origin_bio1 = pd.read_csv('../key_files/snp_origin_bio1_grenenetvcf.csv')

dict_snps = pd.read_csv('../key_files/var_pos_grenenet.csv')

snp_origin_bio1 = pd.concat([dict_snps['id'], snp_origin_bio1],axis=1)

In [ ]:
snp_origin_bio1

In [ ]:
results = results.merge(snp_origin_bio1, left_on = 'snp_id', right_on = 'id')

In [ ]:
results

In [ ]:
results_sign = results[results['pvalue'] < 0.05/len(results)]

In [ ]:
results_sign['slope'].describe()

In [ ]:
sns.histplot(results_sign['slope'])

In [ ]:
results_sign.head()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Assuming snp_origin_beta_binomial_reg is your DataFrame with SNP data

# Create a scatter plot using density for color
fig, ax = plt.subplots(figsize=(12, 6))
sns.scatterplot(
    data=results_sign,
    x='snp_origin_bio1',
    y='slope',
    hue='pvalue',  # Use the density as color
    palette="viridis",  # Color mapping
    edgecolor="none",  # Remove white edge around the dots
    alpha=0.3,  # Set transparency
    ax=ax
)
ax.axhline(0, color='grey', linestyle='--', linewidth=1)
ax.axvline(0, color='grey', linestyle='--', linewidth=1)

# Add a color bar to show the density scale

plt.ylabel('Slope binomial Reg')
# Display the plot
#plt.savefig('snp_origin_across_space.png')

plt.show()


In [ ]:
results_sign.columns

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Assuming snp_origin_beta_binomial_reg is your DataFrame with SNP data

# Create a scatter plot using density for color
fig, ax = plt.subplots(figsize=(12, 6))
sns.scatterplot(
    data=results_sign,
    x='snp_origin_bio1',
    y='2',
    hue='pvalue',  # Use the density as color
    palette="viridis",  # Color mapping
    edgecolor="none",  # Remove white edge around the dots
    alpha=0.3,  # Set transparency
    ax=ax
)
ax.axhline(0, color='grey', linestyle='--', linewidth=1)
ax.axvline(0, color='grey', linestyle='--', linewidth=1)

# Add a color bar to show the density scale

plt.ylabel('Slope binomial Reg')
# Display the plot
#plt.savefig('snp_origin_across_space.png')

plt.show()


In [ ]:
results_sign_most = results_sign.sort_values('pvalue').head(10000)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Assuming snp_origin_beta_binomial_reg is your DataFrame with SNP data

# Create a scatter plot using density for color
fig, ax = plt.subplots(figsize=(12, 6))
sns.scatterplot(
    data=results_sign_most,
    x='snp_origin_bio1',
    y='45',
    hue='pvalue',  # Use the density as color
    palette="viridis",  # Color mapping
    edgecolor="none",  # Remove white edge around the dots
    alpha=0.3,  # Set transparency
    ax=ax,
    legend = False
)
ax.axhline(0, color='grey', linestyle='--', linewidth=1)
ax.axvline(0, color='grey', linestyle='--', linewidth=1)

# Add a color bar to show the density scale

plt.ylabel('Slope binomial Reg')
# Display the plot
#plt.savefig('snp_origin_across_space.png')

plt.show()


In [ ]:
env_site_scaled = pd.read_csv('env_site_scaled.csv').drop_duplicates()

In [ ]:
est = results_sign[['slope', 'pvalue', 'intercept']]

In [ ]:
results_sign

In [ ]:
prob_success_all = {}
for site,env_value in zip(env_site_scaled['site'], env_site_scaled['env_scaled']):
    linear_predictor = env_value * est['slope'] + est['intercept']
    prob_success = 1 / (1 + np.exp(-linear_predictor))
    prob_success_all[site] = prob_success


In [ ]:
pd.DataFrame(prob_success_all)

In [ ]:
prob_success_all = prob_success_all.reset_index(drop=True)

In [ ]:
mask_full_genotype = snp_dict['id'].isin(results_sign['snp_id'])
genotype_counts_red = genotype_counts.loc[mask_full_genotype,:]
genotype_counts_red = genotype_counts_red.reset_index(drop=True)
genotype_counts_red = genotype_counts_red.replace({2: 1, 1: 0.5, 0: 0})

In [ ]:
genotype_counts_red.shape

In [ ]:
prob_success_1_site.shape

In [ ]:
111454048

In [ ]:
unique_sites

In [ ]:
mask_full_genotype = snp_dict['id'].isin(results_sign['snp_id'])
genotype_counts_red = genotype_counts.loc[mask_full_genotype,:]
genotype_counts_red = genotype_counts_red.reset_index(drop=True)
genotype_counts_red = genotype_counts_red.replace({2: 1, 1: 0.5, 0: 0})


go_per_site = {}
for site in unique_sites:
    if site != 33:
        print(site)
        prob_success_1_site = prob_success_all[site]
        genotype_counts_mul = genotype_counts_red.sub(prob_success_1_site, axis=0)   
        genotype_counts_mul = genotype_counts_mul**2
        ## sum over ecotypes 
        go = (genotype_counts_mul.sum(axis=0)) / len(genotype_counts_mul)
        go_per_site[site] = go

In [ ]:
go_per_site_df = pd.DataFrame(go_per_site)

In [ ]:
go_per_site_df.index = go_per_site_df.index.astype(int)

In [ ]:
go_per_site_df.describe()

In [ ]:
go_per_site_df